In [5]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

movies_df = pd.read_csv('movies.csv')
ratings_df = pd.read_csv('ratings.csv')

ratings_matrix = ratings_df.pivot_table(index='userId', columns='movieId', values='rating').fillna(0)

movie_similarity = cosine_similarity(ratings_matrix.T)

movie_similarity_df = pd.DataFrame(movie_similarity, index=ratings_matrix.columns, columns=ratings_matrix.columns)

In [7]:
def get_movie_recommendations(user_id, n_recommendations=5):
    if user_id not in ratings_matrix.index:
        return "User not found in the dataset"
    
    user_ratings = ratings_matrix.loc[user_id]
    
    unrated_movies = user_ratings[user_ratings == 0].index
    
    predicted_ratings = {}
    
    for movie_id in unrated_movies:
        similar_movies = movie_similarity_df[movie_id]
        user_rated_similar = user_ratings[user_ratings > 0].index
        
        if len(user_rated_similar) > 0:
            numerator = sum(similar_movies[other_movie] * user_ratings[other_movie] for other_movie in user_rated_similar)
            denominator = sum(abs(similar_movies[other_movie]) for other_movie in user_rated_similar)
            
            if denominator != 0:
                predicted_ratings[movie_id] = numerator / denominator
    
    recommended_movies = sorted(predicted_ratings.items(), key=lambda x: x[1], reverse=True)[:n_recommendations]
    
    recommendations = []
    for movie_id, pred_rating in recommended_movies:
        movie_title = movies_df[movies_df['movieId'] == movie_id]['title'].iloc[0]
        recommendations.append({
            'movie_id': movie_id,
            'title': movie_title,
            'predicted_rating': round(pred_rating, 2)
        })
    
    return recommendations

In [8]:
user_id = 7  
recommendations = get_movie_recommendations(user_id)
print(f"Movie recommendations for user {user_id}:")
for i, rec in enumerate(recommendations, 1):
    print(f"{i}. {rec['title']} (Predicted rating: {rec['predicted_rating']})")

Movie recommendations for user 7:
1. Get on Up (2014) (Predicted rating: 5.0)
2. Madame Sousatzka (1988) (Predicted rating: 4.88)
3. Audrey Rose (1977) (Predicted rating: 4.79)
4. Friday the 13th Part VII: The New Blood (1988) (Predicted rating: 4.79)
5. Howling II: Your Sister Is a Werewolf (1985) (Predicted rating: 4.79)
